AUA Fall 2022  
DS 110 Statistics 2  
by Mher Movsisyan
# Homework 1 

------
### Problem 1 (15 points)  
How many percent change in `𝑦` could be expected from one unit change in `𝑥`, if the estimated model is (show the derivation).  
$$ ln(y) = \beta_1 + \beta_2 * x^2 $$  

<br>

$$ a) \beta_2/100 $$
$$ b) \beta_2 $$
$$ c) \beta_2 * 100 $$
$$ d) 200\beta_2 * x $$

Answer:  

$$ slope = x\beta_2e^{\beta_2x^2} $$

$$ \eta = \frac{f'(x)}{f(x)} = \frac{2x\beta_2e^{\beta_2x^2}}{e^{\beta_1} * e^{\beta_2x^2}} * 100 = \frac{200x\beta_2}{e^{\beta_1}} $$
$$ \eta_a = \frac{x\beta_2}{50e^{\beta_1}} $$

$$ \eta_b = \frac{x\beta_2}{e^{\beta_1}} $$

$$ \eta_c = \frac{200x\beta_2}{e^{\beta_1} } $$

$$ \eta_d = \frac{600\beta_2x^2}{e^{\beta_1}} $$

Final answer: C  
  
---  

### Problem 2 (15 points)
You have estimated the following simple regression model  
$$ Productivity = 40 + 1.63 Firm\_size - 0.05 Firm\_size^2 $$
What is the elasticity when Firm_size = 35.45? Find the turning point, after which an additional unit increase of Firm size has diminishing impact on Productivity.

Answer:  
The turning point will be when `1.63 * x` will be equal to `0.05 * x^2`. This is at approximately `32.6`.
$$ slope = 1.63 - 0.1x $$
$$ elasticity = slope * x/y = \frac{1.63x - 0.1x^2}{40 + 1.63 x - 0.05 x^2} $$

$$ elasticity(35.45) = -1.94249 $$

### Problem 3 (R, Python) (70 points)
In the file houses.xlsx contains data on 1000 houses sold in US. The data include sale price and
the house size in square feet. Also included is an indicator variable TRADITIONAL indicating
whether the house style is traditional or not.  
a) For the traditional-style houses estimate the linear regression model
$$ Price = \beta_0 + \beta_1SQFT + e $$
Test the null hypothesis that the slope is zero against the alternative that it is positive,
using the 𝛼 = 0.01 level of significance. (15 points)

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tools import add_constant
import scipy.stats as ss

alpha = 0.01

houses = pd.read_excel("Houses.xlsx")
houses

,price,sqft,Traditional
0,66500,741,1
1,66000,741,1
2,68500,790,1
3,102000,2783,1
4,54000,1165,1
...,...,...,...
995,168000,2985,0
996,266900,5140,0
997,235000,3964,0
998,108900,1755,0


In [3]:
# grabbing traditional houses
trad_houses = houses.loc[houses.Traditional == 1]

# Running ols
x = add_constant(trad_houses.sqft)
y = trad_houses.price

model = sm.OLS(y, x)
results = model.fit()
one_tailed_p_value = 1 - ss.t.cdf(results.tvalues[1], results.df_resid)
print(f"P-value: {one_tailed_p_value}")
print("Slope is " + ("significant" if one_tailed_p_value < alpha else "insignificant") + f"ly positive at alpha = {alpha}")

P-value: 0.0
Slope is significantly positive at alpha = 0.01


b) Using the linear model in (a), test the null hypothesis (𝐻0) that the expected price of a
house of 1500 square feet is equal to, or less than, $100,000. What is the appropriate
alternative hypothesis? Use the 𝛼 = 0.01 level of significance. Obtain the p-value of the
test. What is your conclusion? (10 points)

Answer: H1 would be that the expected price of a house of 1500 square feet is greater than $100,000

In [4]:
expected_price_at_1500 = results.get_prediction((1, 1500))
expected_price_at_1500.predicted_mean[0]

82250.36379019442

In [5]:
second_test = results.t_test(([1, 1500], 100_000))
p_value = 1 - ss.t.cdf(second_test.tvalue[0][0], results.df_resid)
print(f"P-value: {p_value}")
print("Null hypothesis was " + ("rejected" if p_value < alpha else "sustained") + f" at alpha = {alpha}")

P-value: 0.9999999998284276
Null hypothesis was sustained at alpha = 0.01


c) Based on the estimated results from part (a), construct a 95% interval estimate of the
expected price of a house of 1500 square feet. (15 points)

In [6]:
se = expected_price_at_1500.se_mean[0]
mean = expected_price_at_1500.predicted_mean[0]
interval = (-1.96 * se + mean, 1.96 * se + mean)
interval

(76805.11119491234, 87695.61638547649)

In [7]:
# shorter version
expected_price_at_1500.summary_frame()[["mean_ci_lower", "mean", "mean_ci_upper"]]

,mean_ci_lower,mean,mean_ci_upper
0,76793.824768,82250.36379,87706.902813


d) For the traditional-style houses, estimate the quadratic regression model
$$ Price = \beta_0 + \beta_1SQFT^2 + e $$
Test the null hypothesis that the marginal effect of an additional square foot of living
area in a home with 1500 square feet of living space is $60 against the alternative that
the effect is less than $60. Use 𝛼 = 0.01 level of significance. Repeat the same test for a
home of 3000 square feet of living space. (15 points)

In [8]:
from statsmodels.formula.api import ols

x = add_constant(trad_houses.sqft**2)
y = trad_houses.price

model = sm.OLS(y, x)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.677
Model:                            OLS   Adj. R-squared:                  0.676
Method:                 Least Squares   F-statistic:                     1214.
Date:                Thu, 08 Sep 2022   Prob (F-statistic):          2.56e-144
Time:                        23:23:52   Log-Likelihood:                -7074.6
No. Observations:                 582   AIC:                         1.415e+04
Df Residuals:                     580   BIC:                         1.416e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.871e+04   2873.195     23.914      0.000    6.31e+04    7.44e+04
sqft           0.0121      0.000     34.839      0.000       0.011       0.013
==============================================================================
Omnibus:                      135.517   Durbin-Watson:                   1.355
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3890.005
Skew:                           0.257   Prob(JB):                         0.00
Kurtosis:                      15.655   Cond. No.                     1.25e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.25e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
at_1500 = results.get_prediction((1, 1500**2))
at_1501 = results.get_prediction((1, 1501**2))
mean_15, se_15 = at_1501.predicted_mean[0] - at_1500.predicted_mean[0], at_1501.se_mean[0] + at_1500.se_mean[0]
mean_15, se_15

(36.201762488504755, 4697.719875224629)

In [10]:
at_3000 = results.get_prediction((1, 3000**2))
at_3001 = results.get_prediction((1, 3001**2))
mean_30, se_30 = at_3001.predicted_mean[0] - at_3000.predicted_mean[0], at_3001.se_mean[0] + at_3000.se_mean[0], 
mean_30, se_30

(72.39146174391499, 4286.783788620144)

In [11]:
ss.ttest_ind_from_stats(mean_15, se_15, 580, 60, se_15, 580)

Ttest_indResult(statistic=-0.08626938007706489, pvalue=0.9312671925559495)

In [12]:
ss.ttest_ind_from_stats(mean_30, se_30, 580, 60, se_30, 580)

Ttest_indResult(statistic=0.04922548000575708, pvalue=0.9607480944332362)

e) For the traditional-style houses, estimate the log-linear regression model  
$$ 𝐿𝑛(𝑃𝑟𝑖𝑐𝑒) = \beta_0 + \beta_1𝑆𝑄𝐹𝑇 + 𝑒 $$  
Test the null hypothesis that the marginal effect of an additional square foot of living
area in a home with 1500 square feet of living space is $60 against the alternative that
the effect is less than $60. Use the 𝛼 = 0.01 level of significance. Repeat the same test
for a home of 3000 square feet of living space. (15 points)

In [13]:
x = add_constant(trad_houses.sqft)
y = np.log(trad_houses.price)

model = sm.OLS(y, x)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     880.4
Date:                Thu, 08 Sep 2022   Prob (F-statistic):          2.12e-118
Time:                        23:23:53   Log-Likelihood:                -113.66
No. Observations:                 582   AIC:                             231.3
Df Residuals:                     580   BIC:                             240.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.7989      0.035    311.473      0.000      10.731      10.867
sqft           0.0004   1.39e-05     29.672      0.000       0.000       0.000
==============================================================================
Omnibus:                      135.860   Durbin-Watson:                   1.337
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              402.676
Skew:                          -1.116   Prob(JB):                     3.63e-88
Kurtosis:                       6.409   Cond. No.                     7.07e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.07e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
at_1500 = results.get_prediction((1, 1500**2))
at_1501 = results.get_prediction((1, 1501**2))
mean_15, se_15 = at_1501.predicted_mean[0] - at_1500.predicted_mean[0], at_1501.se_mean[0] + at_1500.se_mean[0]
mean_15, se_15

(1.240118919378915, 62.64818446622017)

In [15]:
at_3000 = results.get_prediction((1, 3000**2))
at_3001 = results.get_prediction((1, 3001**2))
mean_30, se_30 = at_3001.predicted_mean[0] - at_3000.predicted_mean[0], at_3001.se_mean[0] + at_3000.se_mean[0], 
mean_30, se_30

(2.479824603529778, 250.7038033776562)

In [18]:
ss.ttest_ind_from_stats(mean_15, se_15, 580, np.log(60), se_15, 580)

Ttest_indResult(statistic=-0.7758518728299205, pvalue=0.43799487188678143)

In [19]:
ss.ttest_ind_from_stats(mean_30, se_30, 580, np.log(60), se_30, 580)

Ttest_indResult(statistic=-0.10966839673610433, pvalue=0.9126913663738927)

Please solve more of the last two (3.d, 3.e) in PSS and post on moodle, thanks.